In [2]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

# Load dataset
data = fetch_openml(name='letter', version=1, as_frame=True)
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

results = []
all_histories = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=i)
    acc_history = []

    def objective(trial):
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        C = trial.suggest_loguniform('C', 1e-3, 1e2)

        if kernel in ['rbf', 'poly', 'sigmoid']:
            gamma = trial.suggest_loguniform('gamma', 1e-4, 1e-1)
            clf = svm.SVC(kernel=kernel, C=C, gamma=gamma)
        else:
            clf = svm.SVC(kernel=kernel, C=C)

        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        acc_history.append(acc)
        return acc

    def callback(study, trial):
        acc_history.append(trial.value)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, callbacks=[callback])

    best_trial = study.best_trial
    results.append({
        'Sample': f'S{i+1}',
        'Best Accuracy': round(best_trial.value * 100, 2),
        'Parameters': str(best_trial.params)
    })
    all_histories.append(acc_history)

# Create results DataFrame
df = pd.DataFrame(results)
print(df)

# Plot convergence for the best sample
best_index = df['Best Accuracy'].idxmax()
plt.figure(figsize=(8, 5))
plt.plot(all_histories[best_index])
plt.title('Fitness (bestAccuracy)')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.grid()
plt.show()


[I 2025-04-16 13:57:52,253] A new study created in memory with name: no-name-b593124f-02e3-466e-b6f1-0559b7386368
C:\Users\Admin\AppData\Local\Temp\ipykernel_13868\2546767094.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13868\2546767094.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-4, 1e-1)
[I 2025-04-16 13:58:03,885] Trial 0 finished with value: 0.17616666666666667 and parameters: {'kernel': 'rbf', 'C': 0.020170224030398608, 'gamma': 0.003772044600869464}. Best is trial 0 with value: 0.17616666666666667.
C:\Users\Admin\App

KeyboardInterrupt: 